In [7]:
import requests
import json

def getUserIP(ip):
    url = "https://api.vvhan.com/api/getIpInfo?ip="+str(ip)
    response =requests.get(url)
    if response.status_code==200:
        content=json.loads(response.content)
        print(content)
        if content["success"]==True:
            city=content["info"]["city"]
        else:
            city="未知"
    else:
        city="未知"
    res={"ip":ip,"city":city}
    return res

print(getUserIP("127.0.0.1"))

{'success': True, 'ip': '127.0.0.1', 'info': {'country': '本机地址', 'prov': '-', 'city': '-', 'lsp': '本机地址', 'postcode': '0'}}
{'ip': '127.0.0.1', 'city': '-'}
